# OECD Data Preprocess

In [1]:
import pandas as pd
import numpy as np

In [2]:
# get data
economic_indicator_full = pd.read_csv('data/economic_indicator.csv')
population_full = pd.read_csv('data/population.csv')
cpi_full = pd.read_csv('data/cpi.csv')
gdp_and_production_full = pd.read_csv('data/gdp_and_productivity.csv')
health_full = pd.read_csv('data/health.csv')
ppp_full = pd.read_csv('data/ppp.csv')
productivity_ulc_full = pd.read_csv('data/productivity_ulc.csv')
resident_full = pd.read_csv('data/resident.csv')

In [3]:
# get relevant columns from data
economic_indicator = economic_indicator_full[['Subject', 'Country', 'Time', 'Value']]
population = population_full[['Country', 'Time', 'Value']]
cpi = cpi_full[['Subject', 'Country', 'Time', 'Value']]
gdp_production = gdp_and_production_full[['Subject', 'Country', 'Time', 'Value']]
health = health_full[['Function', 'Country', 'Year', 'Value']]
ppp = ppp_full[['Indicator', 'Country', 'Time', 'Value']]
productivity_ulc = productivity_ulc_full[['Subject', 'Country', 'Time', 'Value']]
resident = resident_full[['Subject', 'Country', 'Time', 'Value']]

Now we check if all the dataset has the same countries.

In [4]:
def assert_same_countries(list1, list2):
    """
    Check if lists have some countries
    """
    try:
        assert len(list1['Country'].unique()) == len(list2['Country'].unique())
    except:
        print("Country not the same")

        if len(list1['Country'].unique()) > len(list2['Country'].unique()):
            print('list 2 missing:')
            for col in list1['Country'].unique():
                if col not in list2['Country'].unique():
                    print(col)
        else:
            print('list 1 missing:')
            for col in list2['Country'].unique():
                if col not in list1['Country'].unique():
                    print(col)

Now we use `assert_same_countries` to see if the data have the same countries.

In [5]:
assert_same_countries(economic_indicator, population)

In [6]:
assert_same_countries(economic_indicator, cpi)

In [7]:
assert_same_countries(economic_indicator, gdp_production)

Country not the same
list 2 missing:
Argentina
Saudi Arabia


In [8]:
assert_same_countries(economic_indicator, health)

Country not the same
list 2 missing:
Argentina
Saudi Arabia


In [9]:
assert_same_countries(economic_indicator, ppp)

Country not the same
list 2 missing:
Argentina
Saudi Arabia
Costa Rica


In [10]:
assert_same_countries(economic_indicator, productivity_ulc)

Country not the same
list 2 missing:
Argentina
Saudi Arabia
Costa Rica


In [11]:
assert_same_countries(economic_indicator, resident)

Country not the same
list 2 missing:
Argentina
Costa Rica


From the output above, we see that some data are missing Argentina, Saudi Arabia, and Costa Rica. Therefore, the three countries will be dropped from all dataset.

In [12]:
df_list = []

for data in [economic_indicator, population, cpi, gdp_production, health, ppp, productivity_ulc, resident]:
    df_list.append(data.loc[(data['Country'] != 'Argentina') & 
                                          (data['Country'] != 'Saudi Arabia') & 
                                          (data['Country'] != 'Costa Rica')])

In [13]:
economic_indicator = df_list[0]
population = df_list[1]
cpi = df_list[2]
gdp_production = df_list[3]
health = df_list[4]
ppp = df_list[5]
productivity_ulc = df_list[6]
resident = df_list[7]

In [14]:
assert_same_countries(economic_indicator, population)
assert_same_countries(economic_indicator, cpi)
assert_same_countries(economic_indicator, gdp_production)
assert_same_countries(economic_indicator, health)
assert_same_countries(economic_indicator, ppp)
assert_same_countries(economic_indicator, productivity_ulc)
assert_same_countries(economic_indicator, resident)

## Economic Indicator

In [15]:
# check if there are null values
economic_indicator.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41670 entries, 0 to 42630
Data columns (total 4 columns):
Subject    41670 non-null object
Country    41670 non-null object
Time       41670 non-null int64
Value      41670 non-null float64
dtypes: float64(1), int64(1), object(2)
memory usage: 1.6+ MB


In [16]:
print("# of subject: {}, # of countries: {}".format(len(economic_indicator['Subject'].unique()),
                                                    len(economic_indicator['Country'].unique())))

# of subject: 26, # of countries: 43


In [17]:
def create_two_dimensional_arr(subject, countries, data):
    """
    """
    avg_value_subject = []
    for subject in subjects:
        averages = []
        for country in countries:
            subject_value_avg = data.loc[(data['Subject'] == subject) & 
                                                       (data['Country'] == country)]['Value'].mean()
            averages.append(subject_value_avg)
        avg_value_subject.append(averages)
    return avg_value_subject

In [18]:
subjects = economic_indicator['Subject'].unique()
countries = sorted(economic_indicator['Country'].unique()) # sort country list by alphabetical order

econ_indi_arr = create_two_dimensional_arr(subjects, countries, economic_indicator)

In [19]:
assert len(econ_indi_arr[0]) == len(economic_indicator['Country'].unique())
assert len(econ_indi_arr) == len(economic_indicator['Subject'].unique())

In [20]:
data = dict(zip(subjects, econ_indi_arr))
avg_economic_indicator = pd.DataFrame(data, index=countries)

In [21]:
avg_economic_indicator.head()

,"Exports in goods, s.a.","Imports in goods, s.a.",Share prices,Consumer prices: all items,"Industrial production, s.a.","Permits issued for dwellings, s.a.","Passenger car registrations, s.a.","Retail trade (Volume), s.a.","Gross domestic product; constant prices, s.a.","GDP Private final consumption expenditure; constant prices, s.a.",...,Overnight interbank rate,Long-term interest rate,"Exchange rates, monthly averages, National currency per US dollar",3 month interbank rate,"Harmonised unemployment rate: all persons, s.a.","Broad money, s.a.","Unit labour cost: Total Economy, s.a.","Total manufacturing, s.a.","Construction, s.a.","Current account as a % of GDP, s.a."
Australia,65.066161,62.010703,4.435165,31.276290,31.347402,26.279443,32.962547,31.039611,2.846152,3.146623,...,3.706842,4.370501,1.311715,4.038026,5.447924,29.365487,31.477237,34.605469,30.277256,-3.954578
Austria,52.938333,54.467084,7.284979,31.701839,32.603948,NaN,34.516842,32.933805,1.550005,1.177416,...,1.193111,2.787935,0.831370,1.544161,5.042105,NaN,31.462773,32.523495,34.861578,2.247881
Belgium,132.521391,127.287131,3.598502,32.057675,33.280718,38.036488,34.359046,31.924394,1.580833,1.368922,...,1.193111,2.957237,0.831370,1.544161,7.606579,NaN,32.369499,32.565987,51.093849,0.899500
Brazil,63.382781,53.096924,11.892222,30.001269,34.055932,NaN,NaN,28.788349,2.286007,2.713431,...,12.631579,NaN,2.587612,NaN,NaN,29.341794,NaN,34.668558,32.901928,-1.619464
Canada,131.896120,127.755152,3.512017,32.056941,33.092914,32.333241,35.071111,31.514421,1.967490,2.766567,...,1.802758,3.175885,1.217234,2.072132,6.941667,29.977154,31.336217,34.354295,30.622559,-1.068944


## Population

In [22]:
population.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71982 entries, 0 to 77003
Data columns (total 3 columns):
Country    71982 non-null object
Time       71982 non-null int64
Value      71982 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.2+ MB


In [23]:
population.head()

,Country,Time,Value
0,Australia,2001,9712875.0
1,Australia,2002,9819727.0
2,Australia,2003,9933140.0
3,Australia,2004,10036771.0
4,Australia,2005,10157211.0


In [103]:
countries = sorted(population['Country'].unique())

In [104]:
mean_values = []

for country in countries:
    lst = population.loc[population['Country'] == country][['Value']]
    aus = [lst[i:i + 18] for i in range(0, len(lst), 18)]
    mean = [aus[i].pct_change().dropna().values.mean() for i in range(len(aus))]
    mean_values.append(mean)

In [105]:
population_mean_dict = dict(zip(countries, mean_values))

In [106]:
avg_population = pd.DataFrame(population_mean_dict)
avg_population = avg_population.T

In [107]:
avg_population.head()

,0,1,2,3,4,5,6,7,8,9,...,83,84,85,86,87,88,89,90,91,92
Australia,0.015409,0.015399,0.015404,0.012788,0.013029,0.012912,0.010611,0.010593,0.010602,0.006907,...,0.014254,-0.000160,0.002136,0.000976,-0.000817,0.019901,-0.001224,-0.001631,0.022263,0.025514
Austria,0.004681,0.006498,0.005567,0.003430,0.004044,0.003745,-0.006905,-0.006279,-0.006584,-0.006874,...,0.012429,-0.001670,0.001448,-0.000265,0.059131,-0.000793,-0.001393,0.053378,-0.000240,0.071006
Belgium,0.005712,0.006472,0.006085,0.003969,0.004259,0.004118,0.005497,0.005562,0.005530,0.002429,...,0.007542,0.001139,0.003637,0.002316,0.035183,-0.001672,-0.000723,0.030503,-0.001192,0.025675
Brazil,0.010273,0.009609,0.009947,-0.009491,-0.009041,-0.009261,-0.010580,-0.009320,-0.009940,-0.009198,...,0.021941,-0.017064,-0.018461,-0.017705,0.004375,-0.030872,-0.031504,-0.030182,0.004035,0.004729
Canada,0.010372,0.010666,0.010517,0.006302,0.006351,0.006327,0.000711,0.000369,0.000536,-0.002076,...,0.019703,0.001331,0.001964,0.001635,0.025217,-0.001391,0.029820,-0.001677,0.027329,-0.001532


## Consumer Price Index

In [119]:
subjects = cpi['Subject'].unique()
countries = sorted(cpi['Country'].unique())

cpi_arr = create_two_dimensional_arr(subjects, countries, cpi)

In [121]:
assert len(cpi_arr[0]) == len(cpi['Country'].unique())
assert len(cpi_arr) == len(cpi['Subject'].unique())

In [123]:
data = dict(zip(subjects, cpi_arr))

In [124]:
avg_cpi_index = pd.DataFrame(data, index=countries)

In [125]:
avg_cpi_index.head()

,CPI: 01-12 - All items,CPI: 01 - Food and non-Alcoholic beverages,CPI: Energy,CPI: All items non-food non-energy
Australia,2.544193,2.565971,4.339597,2.437083
Austria,1.899952,2.273441,2.315837,1.818234
Belgium,1.936742,2.295579,3.315962,1.689575
Brazil,6.320469,7.513241,NaN,NaN
Canada,1.886370,2.469971,2.752541,1.744280


## Level of GDP per capita and Productivity

In [128]:
subjects = gdp_production['Subject'].unique()
countries = sorted(gdp_production['Country'].unique())

gdp_production_arr = create_two_dimensional_arr(subjects, countries, gdp_production)

In [129]:
assert len(gdp_production_arr[0]) == len(gdp_production['Country'].unique())
assert len(gdp_production_arr) == len(gdp_production['Subject'].unique())

In [ ]:
data = dict(zip(subjects, gdp_production_arr))

## Health

## Purchasing Power Parity

## Growth in GDP per capita productivity and ULC

## Residential Property Price Indices